<a href="https://colab.research.google.com/github/MinsooKwak/Kaggle_note/blob/main/image/iceberg/Image_binary_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Kaggle의 Statoil/C-CORE 빙산 분류 데이터셋을 사용하여 이미지 이중분류를 수행합니다.

1. 캐글 데이터셋:

  [Kaggle data | Statoil/C-CORE Iceberg Classifier Challenge](https://www.kaggle.com/c/statoil-iceberg-classifier-challenge)


2. 필사 진행한 캐글 참고 notebook: 

  [필사 참고| Keras Model for Beginners (0.210 on LB)+EDA+R&D ](https://www.kaggle.com/devm2024/keras-model-for-beginners-0-210-on-lb-eda-r-d)

### 데이터 불러오기

먼저 구글 드라이브와 연동시키고 해당 데이터셋이 저장된 폴더로 이동해줍니다.

캐글의 데이터셋을 받아와 사용하기 때문에 캐글 api를 사용할 수 있지만, 구글 드라이브를 통한 방식을 택했습니다.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
cd /content/drive/MyDrive/kaggle/iceberg_classification/input/

/content/drive/MyDrive/kaggle/iceberg_classification/input


In [3]:
ls

sample_submission.csv.7z  test.json.7z  train.json.7z


input data files는 ".../input/" input이라는 directory 안에 있어야 한다 설명하고 있습니다. 

필요한 라이브러리를 import 합니다.

([Pandas 라이브러리에 대해 설명해 놓은 블로그 첨부](https://1000yun.tistory.com/2) )

필사한 notebook에서는 pandas가 데이터 전처리와 csv 파일 import 또는 output에 유용하다고 설명하고 있습니다.

In [4]:
import pandas as pd   # 데이터 처리에 유연한 파이썬 패키지
import numpy as np    # 연산에 필요한 라이브러리

In [5]:
from subprocess import check_output

# subprocess 모듈은 파이썬에서 쉘 명령을 실행할 수 있도록 도와주는 라이브러리이다.
# 단순히 실행만을 원하면 call을 사용하고,
# subprocess에서 출력되는 결과값을 문자열 데이터로 사용하고 싶은 경우 check_output 함수를 사용한다고  한다.
# 결과값이 문자열 데이터로 리턴된다.

```
subprocess.check_output(args, *, stdin=None, stderr=None, shell=False, universal_newlines=False)
```
[subprocess module 참고자료](http://www.incodom.kr/%ED%8C%8C%EC%9D%B4%EC%8D%AC/%EB%9D%BC%EC%9D%B4%EB%B8%8C%EB%9F%AC%EB%A6%AC/subprocess)

In [6]:
print(check_output(["ls","../input"]).decode("utf8"))

sample_submission.csv.7z
test.json.7z
train.json.7z



In [ ]:
# 인코딩이란 문자열을 byte 코드 형식 (utf-8, euc-kr, ascii 코드 등)으로 변환하는 것
# 디코딩이란 utf-8, euc-kr, ascii 형식의 바이트 코드를 문자열로 반환하는 방법을 말한다.

In [7]:
from sklearn.model_selection import train_test_split
from os.path import join as opj                       # 하나 이상의 경로 구성 요소를 결합
from matplotlib import pyplot as plt                  # matplotlib (파이썬 시각화 라이브러리)
from mpl_toolkits.mplot3d import Axes3D               # matplotlib 2D 그래프에 3D 그래프 사용할 수 있도록 해줌
#import pylab                                         # pyplot과 numpy를 단일 네임스페이스로 결합. # 추후 사라질 것으로 pyplot따로 사용 권장
import matplotlib
# matplotlib.pylab의 rcParams 설정을 활용하면, 차트의 크기와 선의 색, 두께등의 기본 값을 설정할 수 있다.
plt.rcParams['figure.figsize']= 10,10
%matplotlib inline

In [ ]:
# sklearn의 model_selection 모듈: 학습 데이터와 테스트셋 데이터를 분리하거나 교차검증 분할 및 평가, estimator의 하이퍼파라미터 튜닝을 위한 다양한 함수와 클래스 제공

([sklearn | model_selection 모듈 설명 블로그](https://injo.tistory.com/10) )
```
- test_size : 전체 데이터 세트에서 테스트 데이터 세트 크기를 얼마로 샘플링할 것인지 결정, 기본값 : 0.25
- train_size : 전체 데이터 세트에서 학습용 데이터 세트 크기를 얼마로 샘플링 할 것인지 결정, 
               test_size를 주로 사용해서 잘 쓰는 파라미터는 아님
- shuffle : 데이터를 분리하기 전에 데이터를 미리 섞을지 결정, 기본값은 True, 데이터를 분산시켜 보다 효율적인 학습/테스트 데이터 세트를 만드는데 사용
- random_state : 난수 값을 지정하면 여러번 다시 수행해도 동일한 결과가 나오게 해줌

반환 값은 튜플 형태: 
(1) 학습용 데이터 피처 데이터 셋 
(2) 테스트용 데이터 피처 데이터셋, 
(3) 학습용 데이터 레이블 데이터 셋, 
(4) 훈련용 데이터 레이블 데이터셋
```

([os.path.join 사용법](https://engineer-mole.tistory.com/188) )
- join()은 디렉토리 구분자 /가 있으면 그것을 root로 본다.
  - 대처법으로는 리스트 사용과 replace 함수 사용 replace("/","")

([from mpl_toolkits.mplot3d import Axes3D | matplotlib 2D --> 3D](https://m.blog.naver.com/PostView.naver?isHttpsRedirect=true&blogId=garden2040&logNo=221410368568) )

([matplotlib | rcParams || default setting, 원상복귀 설명글](https://mirandaherr.tistory.com/18) )
- 전역변수이기 때문에 스타일 변경 전까지 유지

([rcParams 설명](https://koreapy.tistory.com/358) )

### 데이터 압축 풀기

현재 파일이 압축되어 있기 때문에 풀어주는 과정이 필요하다.

```
sample_submission.csv.7z
test.json.7z
train.json.7z
```
파이썬의 extractall() 함수를 통해 압축을 해제할 수 있다 한다. [참고](https://www.delftstack.com/ko/howto/python/python-unzip-file/)
- 작업 디렉토리에
- 다른 디렉토리에 (extractall 함수의 인수명을 설정해주면 됨)
- 조건에 따라 ex) endswith('.txt')
- unpack_archieve 메서드

In [ ]:
# from zipfile import ZipFile
'''
test_file_name = "test.json.7z"
train_file_name = "train.json.7z"

# test file 압축 해제
with ZipFile(test_file_name, 'r') as zip: # 'r'은 파일 읽기 작업
  zip.printdir()
  zip.extractall()                        # extractall은 zip 파일의 내용을 현재 디렉터리에 추출

# train file 압축 해제
with ZipFile(train_file_name,'r') as zip:
  zip.printdir()
  zip.extractall()
'''
# BadZipFile error : File is not a zip file 


여러 파일 압축 해제 예시 
```
with ZipFile('my_files.zip', 'r') as obj_zip:
    FileNames = obj_zip.namelist()
    for fileName in FileNames:
        if fileName.endswith('.txt'):
            zipObj.extract(fileName, 'temp_txt')
```

zipfile 오류가 발생해서 다른 방법을 찾던 중 colab에 구현되어 있는 7z 해제 방법을 찾았다. 다음과 같이 실행하면 압축 해제된 파일이 directory 내에 저장된다.

In [8]:
!7z e sample_submission.csv.7z
!7z e test.json.7z
!7z e train.json.7z


7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.20GHz (406F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan         1 file, 38566 bytes (38 KiB)

Extracting archive: sample_submission.csv.7z
--
Path = sample_submission.csv.7z
Type = 7z
Physical Size = 38566
Headers Size = 178
Method = LZMA2:17
Solid = -
Blocks = 1

  0%    Everything is Ok

Size:       117951
Compressed: 38566

7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.20GHz (406F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan         1 file, 257127394 bytes (246 MiB)

Extracting archive: test.json.7z
--
Path = test.json.7z
Type = 7z
Physical Size = 257127394
Headers Size = 154
Method = LZMA2:24
Solid = -
Blocks = 1

  0%

In [10]:
train = pd.read_json("train.json")
test = pd.read_json("test.json")
submission = pd.read_csv("sample_submission.csv")

### 데이터 확인

In [11]:
train.head(2)

,id,band_1,band_2,inc_angle,is_iceberg
0,dfd5f913,"[-27.878360999999998, -27.15416, -28.668615, -...","[-27.154118, -29.537888, -31.0306, -32.190483,...",43.9239,0
1,e25388fd,"[-12.242375, -14.920304999999999, -14.920363, ...","[-31.506321, -27.984554, -26.645678, -23.76760...",38.1562,0


In [12]:
test.head(2)

,id,band_1,band_2,inc_angle
0,5941774d,"[-15.863251, -15.201077, -17.887735, -19.17248...","[-21.629612, -21.142353, -23.908337, -28.34524...",34.966400
1,4023181e,"[-26.058969497680664, -26.058969497680664, -26...","[-25.754207611083984, -25.754207611083984, -25...",32.615072


In [13]:
submission.head(2)

,id,is_iceberg
0,5941774d,0.5
1,4023181e,0.5


target이 is_iceberg 임을 확인할 수 있다.

## 데이터 설명

- 배 또는 빙산을 포함한 데이터셋
- 모든 데이터셋은 75 * 75 image
----
- id : 이미지의 id
- band1, band2 : 
  - image의 flattened된 데이터
```
Each band has 75x75 pixel values in the list, so the list has 5625 elements.
these are float numbers with unit being dB
```
  - Backscatter(후방산란) 기법을 사용해서 얻어진 데이터

- inc_angle: incidence angle. 즉 이미지가 형성된 앵글
  - 결측치를 포함('na')
    - 데이터 유출 방지를 위해 모든 training data에 포함되어 있음

- ice_berg: ice_berg 1, ship 0
